## Import libraries

In [6]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from tqdm.autonotebook import tqdm


some function

In [20]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
#pc.create_index("rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [4]:
import json
data = json.load(open("full_lawfirm_rag.json"))


In [23]:
import unicodedata

def sanitize_id(law_firm_name):
    # Normalize and encode to ASCII, replacing non-ASCII characters
    return unicodedata.normalize('NFKD', law_firm_name).encode('ascii', 'ignore').decode('ascii')

processed_data = []
client = OpenAI()
for firm in data[0]["firms"]:
    response = client.embeddings.create(
        input=firm["lawFirmName"],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": sanitize_id(firm["lawFirmName"]),  # Sanitize the ID
        "metadata": {
            "firm": firm["lawFirmName"],
            "techCenterDescription": data[0]["techCenterDescription"],
            "rankOverall": firm["rankOverall"]
        }
    })

In [24]:
processed_data[0]

{'values': [-7.610308e-05,
  0.038182452,
  -0.019444535,
  0.0047728065,
  0.0034447487,
  0.01636571,
  0.023595897,
  0.046409473,
  0.0011608677,
  0.003577239,
  0.042952105,
  0.026220469,
  -0.0212994,
  -0.028718857,
  0.009980938,
  0.05228952,
  0.053450387,
  0.0034037398,
  -0.022548594,
  0.037652493,
  0.036264498,
  0.072730884,
  0.018296285,
  0.0076781306,
  0.03454843,
  0.005031478,
  -0.005785411,
  -0.019116463,
  0.05970898,
  -0.03295855,
  0.056579683,
  -0.028895512,
  0.027709408,
  0.02224576,
  -0.00016423281,
  0.020327803,
  0.041892182,
  -0.022283614,
  -0.02465582,
  0.031292956,
  0.006719153,
  -0.020933473,
  0.077172466,
  0.04131175,
  0.034876503,
  -0.01825843,
  -0.0708634,
  0.0168452,
  -0.012308983,
  0.0342456,
  -0.015331024,
  -0.002182936,
  -0.018144866,
  0.05910331,
  -0.05814433,
  0.050447274,
  0.014510847,
  -0.023192119,
  0.010302701,
  -0.004514135,
  -0.012277438,
  -0.03787962,
  -0.0023643216,
  0.067885526,
  -0.013842085,


In [25]:
index = pc.Index("patent")
index.upsert(vectors=processed_data, namespace="ns1")

{'upserted_count': 27}

In [26]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}